In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Basicspyspark').getOrCreate()

In [7]:
df = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])

In [8]:
df.count()

3

In [9]:
df.show()

+---+---+
|  A|  B|
+---+---+
|  1|  4|
|  2|  5|
|  3|  6|
+---+---+



In [10]:
# Returns Column Object
df.A

Column<b'A'>

In [11]:
df.select('A').show()

+---+
|  A|
+---+
|  1|
|  2|
|  3|
+---+



In [12]:
# Adding a new column
df.withColumn('C', df.A+1).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  2|
|  2|  5|  3|
|  3|  6|  4|
+---+---+---+

